In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models

# 1. Load the Dataset
# We split the data: 80% for training, 20% for validation
(train_ds, val_ds), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    with_info=True,
    as_supervised=True
)

# 2. Preprocess the Data
IMG_SIZE = 150
BATCH_SIZE = 32

def format_image(image, label):
    # Normalize pixel values to [0, 1]
    image = tf.cast(image, tf.float32) / 255.0
    # Resize all images to 150x150
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    return image, label

# Apply preprocessing, shuffle, batch, and optimize for performance
train_ds = train_ds.map(format_image).shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.map(format_image).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# 3. Design the CNN Model
model = models.Sequential([
    # First Convolutional Block
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D(2, 2),

    # Second Convolutional Block
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    # Third Convolutional Block
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    # Flatten and Dense Layers for Classification
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5), # Dropout prevents overfitting
    layers.Dense(1, activation='sigmoid') # Binary output (0 = cat, 1 = dog)
])

# 4. Compile the Model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# 5. Train the Model
EPOCHS = 10
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds
)

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

def predict_custom_image(model, img_path):
    # 1. Load the image and resize it to match the model's input shape
    img = image.load_img(img_path, target_size=(150, 150))

    # 2. Convert the image to a numpy array
    img_array = image.img_to_array(img)

    # 3. Normalize the pixel values (just like we did for the training data)
    img_array = img_array / 255.0

    # 4. Expand dimensions
    # The model expects a batch of images: (batch_size, height, width, channels)
    # We add an extra dimension to make it (1, 150, 150, 3)
    img_array = np.expand_dims(img_array, axis=0)

    # 5. Make the prediction
    prediction = model.predict(img_array)

    # 6. Interpret the output
    # Since we used a sigmoid activation, the output is a single probability
    probability = prediction[0][0]

    if probability >= 0.5:
        print(f"Prediction: DOG (Probability score: {probability:.4f})")
    else:
        print(f"Prediction: CAT (Probability score: {probability:.4f})")

# --- Test it out ---
# Replace this string with the actual location of your image
my_image_path = '/content/dog.jpg.jpeg'

# Call the function
predict_custom_image(model, my_image_path)